In [1]:
import numpy as np

class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size, lr=0.01):
        self.hidden_size = hidden_size
        self.lr = lr
        
        # Xavier initialization
        self.Wx = np.random.randn(hidden_size, input_size) * 0.1
        self.Wh = np.random.randn(hidden_size, hidden_size) * 0.1
        self.Wy = np.random.randn(output_size, hidden_size) * 0.1
        
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        """Forward pass through sequence"""
        h_prev = np.zeros((self.hidden_size, 1))
        self.cache = []

        outputs = []
        for x in inputs:
            x = x.reshape(-1, 1)
            
            h = np.tanh(self.Wx @ x + self.Wh @ h_prev + self.bh)
            y = self.Wy @ h + self.by
            
            self.cache.append((x, h_prev, h))
            h_prev = h
            outputs.append(y)

        return outputs

    def backward(self, dy_list):
        """Backpropagation Through Time"""
        dWx = np.zeros_like(self.Wx)
        dWh = np.zeros_like(self.Wh)
        dWy = np.zeros_like(self.Wy)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)
        
        dh_next = np.zeros((self.hidden_size, 1))

        for t in reversed(range(len(self.cache))):
            x, h_prev, h = self.cache[t]
            dy = dy_list[t]

            dWy += dy @ h.T
            dby += dy

            dh = self.Wy.T @ dy + dh_next
            dh_raw = (1 - h**2) * dh

            dWx += dh_raw @ x.T
            dWh += dh_raw @ h_prev.T
            dbh += dh_raw

            dh_next = self.Wh.T @ dh_raw

        # Gradient descent update
        for param, dparam in zip(
            [self.Wx, self.Wh, self.Wy, self.bh, self.by],
            [dWx, dWh, dWy, dbh, dby]
        ):
            param -= self.lr * dparam


In [5]:
import pandas as pd

df = pd.read_csv("/kaggle/input/poems-100/poems-100.csv")
print(df.columns)
print(df.head())



Index(['text'], dtype='object')
                                                text
0  O my Luve's like a red, red rose\nThat’s newly...
1  The rose is red,\nThe violet's blue,\nSugar is...
2  How do I love thee? Let me count the ways.\nI ...
3  Had I the heavens' embroidered cloths,\nEnwrou...
4  I.\n    Enough! we're tired, my heart and I.\n...


In [6]:
df.shape

(100, 1)

In [7]:
text_data = " ".join(df["text"].astype(str)).lower()


In [8]:
# text_data

In [9]:
tokens = text_data.split()

print("Total words:", len(tokens))


Total words: 24734


In [10]:
import re
tokens = re.findall(r"\b\w+\b", text_data.lower())


In [11]:
vocab = sorted(set(tokens))
vocab_size = len(vocab)

word_to_ix = {w: i for i, w in enumerate(vocab)}
ix_to_word = {i: w for w, i in word_to_ix.items()}

print("Vocabulary size:", vocab_size)


Vocabulary size: 5158


In [12]:
import numpy as np

def one_hot(idx, vocab_size):
    vec = np.zeros(vocab_size)
    vec[idx] = 1
    return vec


In [36]:
sequence_length = 5

inputs = []
targets = []

for i in range(len(tokens) - sequence_length):
    seq = tokens[i:i+sequence_length]
    target = tokens[i+1:i+sequence_length+1]

    inputs.append([one_hot(word_to_ix[w], vocab_size) for w in seq])
    targets.append([word_to_ix[w] for w in target])

import torch

# X = torch.tensor(inputs, dtype=torch.float32)
Y = torch.tensor(targets, dtype=torch.long)
X = torch.tensor([[word_to_ix[w] for w in tokens[i:i+sequence_length]]
                  for i in range(len(tokens)-sequence_length)], dtype=torch.long)


print(X.shape, Y.shape)


torch.Size([25503, 5]) torch.Size([25503, 5])


In [37]:
class PoetryRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size=64, embed_dim=128):
        super().__init__()

        self.embed = nn.Embedding(vocab_size, embed_dim)

        self.rnn = nn.RNN(
            input_size=embed_dim,
            hidden_size=hidden_size,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)   # <-- key fix
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden


In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [41]:
model = PoetryRNN(vocab_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

epochs = 100


In [42]:
model

PoetryRNN(
  (embed): Embedding(5158, 128)
  (rnn): RNN(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=5158, bias=True)
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PoetryRNN(vocab_size).to(device)

batch_size = 32

for epoch in range(epochs):
    total_loss = 0

    for i in range(0, len(X), batch_size):
        xb = X[i:i+batch_size].to(device)
        yb = Y[i:i+batch_size].to(device)

        optimizer.zero_grad()

        output, _ = model(xb)

        loss = criterion(
            output.reshape(-1, vocab_size),
            yb.reshape(-1)
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 6854.1726
Epoch 2, Loss: 6854.1726
Epoch 3, Loss: 6854.1726
Epoch 4, Loss: 6854.1726
Epoch 5, Loss: 6854.1726
Epoch 6, Loss: 6854.1726
Epoch 7, Loss: 6854.1726
Epoch 8, Loss: 6854.1726
Epoch 9, Loss: 6854.1726
Epoch 10, Loss: 6854.1726
Epoch 11, Loss: 6854.1726
Epoch 12, Loss: 6854.1726
Epoch 13, Loss: 6854.1726
Epoch 14, Loss: 6854.1726
Epoch 15, Loss: 6854.1726
Epoch 16, Loss: 6854.1726
Epoch 17, Loss: 6854.1726
Epoch 18, Loss: 6854.1726
Epoch 19, Loss: 6854.1726
Epoch 20, Loss: 6854.1726
Epoch 21, Loss: 6854.1726
Epoch 22, Loss: 6854.1726
Epoch 23, Loss: 6854.1726
Epoch 24, Loss: 6854.1726
Epoch 25, Loss: 6854.1726
Epoch 26, Loss: 6854.1726
Epoch 27, Loss: 6854.1726
Epoch 28, Loss: 6854.1726
Epoch 29, Loss: 6854.1726
Epoch 30, Loss: 6854.1726
Epoch 31, Loss: 6854.1726
Epoch 32, Loss: 6854.1726
Epoch 33, Loss: 6854.1726
Epoch 34, Loss: 6854.1726
Epoch 35, Loss: 6854.1726
Epoch 36, Loss: 6854.1726
Epoch 37, Loss: 6854.1726
Epoch 38, Loss: 6854.1726
Epoch 39, Loss: 6854.

In [ ]:
def generate_poem(start_words, length=20):
    model.eval()

    words = start_words.lower().split()
    hidden = None

    for _ in range(length):

        seq = words[-sequence_length:]
        seq = [one_hot(word_to_ix.get(w, 0), vocab_size) for w in seq]

        x = torch.tensor([seq], dtype=torch.float32)

        with torch.no_grad():
            output, hidden = model(x, hidden)

        probs = torch.softmax(output[0, -1], dim=0)
        idx = torch.argmax(probs).item()

        words.append(ix_to_word[idx])

    return " ".join(words)


print(generate_poem("love is"))
